# A/B Testing in PyMC: part 2

<div class="alert alert-warning">
<h2>Goal of this session:</h2>

In this hands-on session, you will continue working with the synthetic dataset from yesterday. Your goal is to be able to perform Bayesian inference using the PyMC library. The insight gained from this session will help you understand:

- How to perform Bayesian A/B testing on data with Value conversions
- How to interpret the results

</div>


Let's practice what you have learned this morning. You will be using the [PyMC library](https://www.pymc.io/welcome.html) to do Bayesian inference and explore posterior distributions. 

You only workout the case of two variants A and B with **Value conversions** using the data from yesterday. In the first part of this hands-on session, you compared A and B variants in terms of how many conversions they generate, and estimated the relative uplift of conversions between the two. Now you will be comparing A and B variants in terms of how much revenue they generate, and estimate the relative uplift of revenue between the two.

Below we put this setup into `PyMC` framework and find the posteriors. Let's start by importing the necessary libraries.

In [ ]:
# Load libraries
import matplotlib.pyplot as plt
import arviz as az
import pymc as pm

## Your First Try

Remember the example with synthetic data where we had a website with two variants A and B. Let's assume that we have a Bernoulli likelihood for each variant, and we can model the conversion rate $\theta$ as Beta distribution for each variant. To start with, let's assume that we identical priors for both variants with $\alpha = 1$ and $\beta = 1$.

Note that according to our data, variant B's visitors are more likely to sign up: $$\theta_B \approx 51\% > \theta_A \approx 50\%$$

But their mean payment is smaller than variant A's visitors, because from data we observed that: 

$$\frac{1}{\lambda_B} \approx 55 < \frac{1}{\lambda_A} \approx 61$$

Remember that the expected revenue per visitor was $\frac{\theta}{\lambda}$. So, variant B has smaller expected revenue per visitor than variant A:

$$ 0.51 \times 55 <  0.50 \times 61$$

Therefore, the relative uplift should be negative: 

$$\mathrm{uplift} = \frac{\frac{\theta_B}{\lambda_B}}{\frac{\theta_A}{\lambda_A}} - 1 = \frac{0.51 \times 55}{0.50 \times 61} - 1 = -0.079$$


Let's assume that our prior for $\lambda$ is Gamma distributed with $\alpha = 1000$ and $\beta = 30000$.

<div class="alert alert-success">

**Remarke on the notations**: we should be mindful that $\theta$ and $\lambda$ denote the true and unobserable parameters, and the observed values (e.g. conversion rate of 51\%) should be denoted differently, e.g. by $\hat{\theta}$. For simplicity we keep the same notation for both the true and observed values.

</div>

In [ ]:
# as before we have variants A and B
variants  = ['A', 'B']

# let's define prior's parameters for the conversion rates
conv_alpha, conv_beta = [1, 1]

# let's define prior's parameters for the average payment 
payment_alpha, payment_beta = [1000, 30000]

Now, replace the `< >` with the correct values from the data:

In [ ]:
# number of visitors for each variant 
visitors    = [< >, < >]

# number of conversions (sign ups) for each variant
conversion = [< >, < >]

# sum of payments for each variant
total_revenue = [< >, < >]

The model, defined in the next cell, consist of four main components:
- Prior
- Likelihood
- Uplift (difference between A and B)
- Posterior

Let's run the code

In [ ]:
with pm.Model() as example_model:

    #------------------------------------------------conversion rate model
    # Priors for unknown model parameters
    theta = pm.Beta("theta", 
                    alpha = conv_alpha, 
                    beta  = conv_beta, 
                    shape = 2)
    
    # Likelihood of observations
    converted = pm.Binomial("converted", 
                            n        = visitors,      # total visitors
                            observed = conversion,     # total visitors converted
                            p        = theta,         # chance they convert
                            shape    = 2)  
    
    #------------------------------------------------revenue model
    # Priors for unknown model parameters
    lamda = pm.Gamma( "lamda", 
                    alpha = payment_alpha,
                    beta  = payment_beta,
                    shape = 2)
    
    # Likelihood of observations
    revenue = pm.Gamma("revenue", 
                        alpha    = conversion,            # total visitors converted
                        observed = total_revenue, 
                        beta     = lamda, 
                        shape    = 2)        
    
    # get the revenue per visitor
    revenue_per_visitor = pm.Deterministic("revenue_per_visitor", theta / lamda)

    #------------------------------------------------relative uplifts
    theta_uplift = pm.Deterministic(f"theta uplift", theta[1] / theta[0] - 1)
    lamda_uplift = pm.Deterministic(f"lamda uplift", (1 / lamda[1]) / (1 / lamda[0]) - 1)
    uplift       = pm.Deterministic(f"uplift", revenue_per_visitor[1] / revenue_per_visitor[0] - 1)

    #------------------------------------------------posterior
    # draw posterior samples
    trace = pm.sample(draws=5000, return_inferencedata=True)

In [ ]:
# Check the unobserved random variables:
example_model.unobserved_RVs    

In [ ]:
# Check the observed (synthetic) random variable:
example_model.observed_RVs  

In [ ]:
# Check the deterministic random variable:
example_model.deterministics

In [ ]:
# check the trace, i.e. posterior distribution of the conversion rates
trace

In [ ]:
# find out the shape of all trace.posterior variables
for var in trace.posterior:
    print(var, trace.posterior[var].shape)

In [ ]:
# plot all the three uplifts 
fig, axes = plt.subplots(1,3, figsize=(14,3), sharey=True)
fig.suptitle("Posteriors", fontsize=14)

for i, var in enumerate(["theta uplift", "lamda uplift", "uplift"]):
    az.plot_posterior(trace.posterior[var], ax=axes[i],)
    axes[i].axvline(x=0, color="red")

plt.tight_layout()
plt.show()

<div class="alert alert-info">
<h4>Task 1</h4>

Check out the range of values posterior distributions can take, especially the mean and the values covered by HDI. Then answer the following questions:

- What do you learn from the posterior distribution of uplift for $\theta$?

- What do you learn from the posterior distribution of uplift for $\lambda$?

- What do you learn from the posterior distribution of uplift for $\frac{\theta}{\lambda}$? did you expect its HDI to include zero?
<br>
</div>



write your answer here

Let's continue by exploring the posterior distributions.

In [ ]:
# plot all posterior distributions
az.plot_trace(trace, compact=True, combined=True)
plt.tight_layout();

<div class="alert alert-info">
<h4>Task 2</h4>

Change the prior for $\lambda$ which is curently `Gamma(1000,30000)` and re-do the previous task.
    
<br>
</div>



write your answer here